In [1]:
import numpy as np
from gym import Env
from gym.spaces import Box, Discrete
import random

In [132]:
class CustomEnv(Env):
    def __init__(self):
        self.action_space = Discrete(3)
        self.observation_space = Box(low=np.array([0]), high=np.array([100]))
        self.state = 32 + random.randint(-3,3)
        self.shower_length = 60
        
    def step(self, action):
        self.state += action -1 
        self.shower_length -= 1 
        
        if self.state >=37 and self.state <=40: 
            reward =1 
        else: 
            reward = -1 
        
        if self.shower_length <= 0: 
            done = True
        else:
            done = False
        
        info = {}
        
        # Return step information
        return self.state, reward, done, info
    
    def reset(self):
        self.state = 32 + random.randint(-3,3)
        self.shower_length = 60 
        return self.state
    
    def render(self, mode = "human"):
        print(self.state)

In [133]:
env = CustomEnv()

In [58]:
env.observation_space.sample()

array([70.73981], dtype=float32)

In [59]:
env.action_space.sample()

2

In [91]:
episodes = 20 #20 shower episodes
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
        #env.render()
    print('Episode:{} Score:{}'.format(episode, score))

Episode:1 Score:26
Episode:2 Score:20
Episode:3 Score:-28
Episode:4 Score:-42
Episode:5 Score:-4
Episode:6 Score:-48
Episode:7 Score:34
Episode:8 Score:-52
Episode:9 Score:6
Episode:10 Score:6
Episode:11 Score:28
Episode:12 Score:0
Episode:13 Score:-28
Episode:14 Score:8
Episode:15 Score:-60
Episode:16 Score:-14
Episode:17 Score:-2
Episode:18 Score:-60
Episode:19 Score:-18
Episode:20 Score:10


In [18]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [92]:
states = env.observation_space.shape
actions = env.action_space.n

In [62]:
actions

3

In [63]:
states

(1,)

In [22]:
def build_model(states, actions):
    model = Sequential()    
    model.add(Dense(24, activation='relu', input_shape=states))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [134]:
del model

In [135]:
model = build_model(states, actions)

In [66]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 24)                48        
                                                                 
 dense_10 (Dense)            (None, 24)                600       
                                                                 
 dense_11 (Dense)            (None, 3)                 75        
                                                                 
Total params: 723
Trainable params: 723
Non-trainable params: 0
_________________________________________________________________


In [52]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [53]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [152]:
dqn = build_agent(model, actions)
dqn.compile(Adam(learning_rate=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=2000, visualize=False, verbose=1)

2022-12-13 16:49:16.293691: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-12-13 16:49:16.358484: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-12-13 16:49:16.450434: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


Training for 2000 steps ...
Interval 1 (0 steps performed)
    1/10000 [..............................] - ETA: 31:01 - reward: -1.0000

2022-12-13 16:49:16.601770: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-12-13 16:49:16.680912: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-12-13 16:49:16.698455: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-12-13 16:49:16.811203: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-12-13 16:49:16.976217: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-12-13 16:49:17.058137: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-12-13 16:49:17.080548: I tensorflow/core/grappler/optimizers/cust

   22/10000 [..............................] - ETA: 8:13 - reward: -1.0000

2022-12-13 16:49:17.565844: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


 1998/10000 [====>.........................] - ETA: 2:10 - reward: -1.0000done, took 32.884 seconds


In [153]:
results = dqn.test(env, nb_episodes=1, visualize=True)
print(np.mean(results.history['episode_reward']))

Testing for 1 episodes ...
30
29
28
27
26
25
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
-1
-2
-3
-4
-5
-6
-7
-8
-9
-10
-11
-12
-13
-14
-15
-16
-17
-18
-19
-20
-21
-22
-23
-24
-25
-26
-27
-28
-29
Episode 1: reward: -60.000, steps: 60
-60.0
